# Закроют ли тему на StackOverflow?

#### Попов Артём 317 группа

## Описание задания:

Миллионы разработчиков пользуются StackOverflow каждый день, чтобы находить ответы на свои вопросы. Крайне важно поддерживать качество контента на высоком уровне, чтобы оправдать их доверие и сохранить репутацию.

Порядка 6% от всех новых тем закрываются по одной из причин: оффтопик, неконструктивный вопрос, отсутствие вопроса, слишком частный вопрос. Ваша задача — предсказать по заголовку и тексту темы, будет ли она закрыта. Также предоставляются некоторые данные об авторе темы в момент ее создания.

## Описание решения

Вся суть решения описана в комментариях к коду.

#### Подключение необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import csv
import re

import nltk
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize

import scipy.sparse as sps
import time

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cross_validation import KFold

#### Считывание данных

In [2]:
train_path = 'data/train-contest.csv'
data = pd.read_csv(train_path)

test_path = 'data/test-contest-second.csv'
test = pd.read_csv(test_path)

#### Предобработка данных

1. Данные делятся на код и текст (кодом считаются все строки, которые начинаются с символа табуляции или минимум с четырёх пробелов). Дополнительно первое предложение и последнее запоминаются. Из данных удаляются все символы переноса строки (их количество фиксируется и является одним из признаков (количество строк в коде))

2. Из данных удаляются все символы с неподходящей кодировкой (параллельно генерируются признаки - количество символов с неправильной кодировкой и т.п.)

3. Nan в данных заменяются на пустые строки.

4. Из данных удаляются все : и | для перевода в формат VowpalWabbit

In [3]:
def divide_to_text_and_code(s):
    list_grams = s.split('\n')
    code = ''
    text = ''
    numb_line_code = 0
    numb_line_text = 0
    flag = 0
    first_code = 0
    first_text = 0
    list_grams = filter(lambda x: len(x) >= 4, list_grams)
    for line in list_grams:
        if line[0] == '\t' or line[0:4].isspace():
            if flag == 0:
                flag = 1
                first_code = 1
            code = code + line + ' '
            numb_line_code += 1 
            continue
        if flag == 0:
            flag = 1
            first_text = 1
        text = text + line + ' '
        numb_line_text += 1
    return (text, numb_line_text, code, numb_line_code, first_text, first_code)


def count_not_ascii(s):
    return sum(list(ord(c) >= 128 for c in s))


def delete_not_ascii(s):
    return ''.join(list(c for c in s if ord(c) < 128))


def post_proc(s):
    temp = re.sub("\r|\n", " ", s)
    return temp


def del_colon(s):
    temp = s.replace(":", ".")
    temp = temp.replace("|", ".")
    return temp


def processing(data): 
    # признаки, связанные с не той кодировкой
    data['NotAsciiBody'] = data.BodyMarkdown.apply(lambda x: count_not_ascii(x))
    data['NotAsciiTitle'] = data.Title.apply(lambda x: count_not_ascii(x))
    data['BodyMarkdown'] = data.BodyMarkdown.apply(lambda x: delete_not_ascii(x))
    data['Title'] = data.Title.apply(lambda x: delete_not_ascii(x))
    
    big_col = data.BodyMarkdown.apply(lambda x: divide_to_text_and_code(x))
    data['Text'] = big_col.apply(lambda x: x[0])
    data['Code'] = big_col.apply(lambda x: x[2])
    data['NLinesInText'] = big_col.apply(lambda x: x[1])
    data['NLinesInCode'] = big_col.apply(lambda x: x[3])
    data['FirstIsText'] = big_col.apply(lambda x: x[4])
    data['FirstIsCode'] = big_col.apply(lambda x: x[5])

    
    data['BodyMarkdown'] = data.BodyMarkdown.apply(lambda x: del_colon(post_proc(x)))
    data['Title'] = data.Title.apply(lambda x: del_colon(post_proc(x)))
    data['Text'] = data.Text.apply(lambda x: del_colon(post_proc(x)))
    data['Code'] = data.Code.apply(lambda x: del_colon(post_proc(x)))
    
    data['Tag1'] = data['Tag1'].fillna('')
    data['Tag2'] = data['Tag2'].fillna('')
    data['Tag3'] = data['Tag3'].fillna('')
    data['Tag4'] = data['Tag4'].fillna('')
    data['Tag5'] = data['Tag5'].fillna('')
    return data



In [4]:
data = processing(data)
test = processing(test)

#### Преобразование данных в формат TF-IDF

Берутся отдельно модели TF-IDF по тексту, коду, отдельно по каждому тэгу, заголовку и по совмещённому коду и тексту. В некоторых случаях берутся биграммы.

In [1]:
def itdf(data, test):
    vect = TfidfVectorizer(min_df=10)
        
    data_itdf10 = vect.fit_transform(data.BodyMarkdown.apply(lambda x: x.lower()))
    test_itdf10 = vect.transform(test.BodyMarkdown.apply(lambda x: x.lower()))

    data_itdf3 = vect.fit_transform(data.Title.apply(lambda x: x.lower()))
    test_itdf3 = vect.transform(test.Title.apply(lambda x: x.lower()))

    
    vect = TfidfVectorizer(min_df=10, ngram_range=(1, 2))

    data_itdf1 = vect.fit_transform(data.Text.apply(lambda x: x.lower()))
    test_itdf1 = vect.transform(test.Text.apply(lambda x: x.lower()))
    
    data_itdf2 = vect.fit_transform(data.Code.apply(lambda x: x.lower()))
    test_itdf2 = vect.transform(test.Code.apply(lambda x: x.lower()))

    
    vect = TfidfVectorizer(min_df=2)
    
    data_itdf4 = vect.fit_transform(data.Tag1)
    test_itdf4 = vect.transform(test.Tag1)
    
    data_itdf5 = vect.fit_transform(data.Tag2)
    test_itdf5 = vect.transform(test.Tag2)
    
    data_itdf6 = vect.fit_transform(data.Tag2)
    test_itdf6 = vect.transform(test.Tag2)
    
    data_itdf7 = vect.fit_transform(data.Tag3)
    test_itdf7 = vect.transform(test.Tag3)
    
    data_itdf8 = vect.fit_transform(data.Tag4)
    test_itdf8 = vect.transform(test.Tag4)
    
    data_itdf9 = vect.fit_transform(data.Tag5)
    test_itdf9 = vect.transform(test.Tag5)    
    
    
    data_itdf = sps.hstack((data_itdf1, data_itdf2, data_itdf3, data_itdf4, data_itdf5, data_itdf6, 
                           data_itdf7, data_itdf8, data_itdf9, data_itdf10))
    test_itdf = sps.hstack((test_itdf1, test_itdf2, test_itdf3, test_itdf4, test_itdf5, test_itdf6,
                           test_itdf7, test_itdf8, test_itdf9, test_itdf10))
    return (data_itdf, test_itdf)

In [ ]:
data_itdf, test_itdf = itdf(data, test)

### Генерация признаков

Можно условно поделить признаки на несколько категорий:

1. Длина в символах конкретной сущности (текста, кода, первого предложения, последнего предложения, каждого из тэгов, заголовка)

2. Наличие конкретных сущностей

3. Длина в словах конкретных сущностей

4. Признаки связанные с предложениями (количество предложений, количество вопросительных предлоений, отношение количества вопросительных ко всем и тп.)

5. Признаки учитывающие конкретные символы, конкретные слова

Не все из признаков, сгенерированных ниже, использовались при обучении модели.

In [ ]:
def detect_first_letter(s):
    if (len(s) == 0 or not s[0].isupper()):
        return 0
    else:
        return 1

    
def detect_first_letter_list(s):
    return sum(map(lambda x: detect_first_letter(x), s))


def detect_last_char(s, my_char):
    if (len(s) == 0 or s[-1] != my_char):
        return 0
    else:
        return 1

    
def detect_last_char_list(s, my_char):
    return sum(map(lambda x: detect_last_char(x, my_char), s))
    
    
def count_mean(s, tokenizer):
    l = [len(seq) for seq in tokenizer(s)]
    return np.mean(np.array(l))


def count_mean_of_mean(s):
    l = np.mean(np.array(map(lambda y: np.mean(np.array(map(lambda x: len(x), word_tokenize(y)))), sent_tokenize(s))))
    return l


def exist(x):
    if x > 0:
        return 1.0
    else:
        return 0.0

    
def sent(x):
    list_sent = sent_tokenize(x)
    list_sent = list(filter(lambda x: len(x) >= 4 , list_sent))
    list_sent = list(map(lambda x: str.strip(x), list_sent))
    return list_sent


def run_len_encode_vec(x):
    y = np.hstack(([-1], x, [0]))
    temp = np.hstack(([0], np.diff(np.where(np.diff(y) != 0))[0]))
    temp = np.cumsum(temp)
    return y[np.diff(y) != 0][1:], temp

In [5]:
def feature_add(data):
# создание новых сущностей
    sent_text = data.Text.apply(lambda x: sent(x))
    sent_text = sent_text.apply(lambda x: [''] if len(x) == 0 else x)
    data['FirstText'] = sent_text.apply(lambda x: x[0])
    data['LastText'] = sent_text.apply(lambda x: x[-1])    
    
# длина
    data['TextLen'] = data.Text.apply(lambda x: len(x))
    data['CodeLen'] = data.Code.apply(lambda x: len(x))
    data['TitleLen'] = data.Title.apply(lambda x: len(x))
    data['BodyLen'] = data.BodyMarkdown.apply(lambda x: len(x))
    
    data['Tag1Len'] = data.Tag1.apply(lambda x: len(x))
    data['Tag2Len'] = data.Tag2.apply(lambda x: len(x))
    data['Tag3Len'] = data.Tag3.apply(lambda x: len(x))
    data['Tag4Len'] = data.Tag4.apply(lambda x: len(x))
    data['Tag5Len'] = data.Tag5.apply(lambda x: len(x))
    data['AllTagLen'] = data['Tag1Len'] + data['Tag2Len'] + data['Tag3Len'] + data['Tag4Len'] + data['Tag5Len']
    
    data['FirstTextLen'] = data.FirstText.apply(lambda x: len(x))
    data['LastTextLen'] = data.LastText.apply(lambda x: len(x))
    

# существование 
    data['TextExist'] = data.TextLen.apply(lambda x: exist(x))
    data['CodeExist'] = data.CodeLen.apply(lambda x: exist(x))
    data['Tag1Exist'] = data.Tag1Len.apply(lambda x: exist(x))
    data['Tag2Exist'] = data.Tag2Len.apply(lambda x: exist(x))
    data['Tag3Exist'] = data.Tag3Len.apply(lambda x: exist(x))
    data['Tag4Exist'] = data.Tag4Len.apply(lambda x: exist(x))
    data['Tag5Exist'] = data.Tag5Len.apply(lambda x: exist(x))
    
    
# признаки, связанные со словами
    data['NWordsInText'] = data.Text.apply(lambda x: len(x.split()))
    data['NWordsInTitle'] = data.Title.apply(lambda x: len(x.split())) 
    data['NWordsInCode'] = data.Code.apply(lambda x: len(x.split())) 
    data['NWordsInBody'] = data.BodyMarkdown.apply(lambda x: len(x.split()))
    
    
# признаки, связанные с предложениями
    data['NSentenceInText'] = sent_text.apply(lambda x: len(x))
    data['FirstLetterInSentence'] = sent_text.apply(lambda x: detect_first_letter_list(x))
    data['NQuest'] = sent_text.apply(lambda x: detect_last_char_list(x, '?'))
    data['NDot'] = data.Text.apply(lambda x: detect_last_char_list(x, '.'))
    data['NExcl'] = data.Text.apply(lambda x: detect_last_char_list(x, '!'))
    data['RatioQuestAll'] = data['NQuest'] / (data['NSentenceInText'] + 1)
    data['RatioDotAll'] = data['NDot'] / (data['NSentenceInText'] + 1)
    data['RatioExclAll'] = data['NExcl'] / (data['NSentenceInText'] + 1)


# признаки касающиеся особых символов и слова
    data['DigitCount'] = data.Text.apply(lambda x: sum(c.isdigit() for c in x))
    data['ArithmCountText'] = data.Text.apply(lambda x: x.count('+') + x.count('-') + x.count('*'))
    data['ArithmCountCode'] = data.Code.apply(lambda x: x.count('+') + x.count('-') + x.count('*'))
    data['EqCountText'] = data.Text.apply(lambda x: x.count('='))
    data['EqCountCode'] = data.Code.apply(lambda x: x.count('='))
    data['Thanks'] = data.LastText.apply(lambda x: x.lower().count('thanks'))
    data['I'] = data.FirstText.apply(lambda x: x.lower().count('i '))
    data['UrlCount'] = data.Text.apply(lambda x: x.count('http'))
    

# отношения признаков
    data['RatioTextCode'] = data['TextLen'] / (data['CodeLen'] + 1)
    data['RatioFirstText'] = data['FirstTextLen'] / (data['TextLen'] + 1) 
    data['RatioLastText'] = data['LastTextLen'] / (data['TextLen'] + 1)
    data['RatioTextNWords'] = (data['TextLen']) / (data['NWordsInText'] + 1)
    data['RatioTextNSentence'] = data['TextLen'] / (data['NSentenceInText'] + 1)
    data['RatioNWordsNSentence'] = data['NWordsInText'] / (data['NSentenceInText'] + 1)
    data['RatioCodeNLines'] = data['CodeLen'] / (data['NLinesInCode'] + 1)
    data['RatioNotAsciiBody'] = data['NotAsciiBody'] / (data['BodyLen'] + 1)
    data['RatioNotAsciiTitle'] = data['NotAsciiTitle'] / (data['TitleLen'] + 1)

# что-то странное
    data['EqCountTitle'] = data.BodyMarkdown.apply(lambda x: x.count('='))
    data['ArithmCount'] = data.BodyMarkdown.apply(lambda x: x.count('+') + x.count('-') + x.count('*'))
    data['ArithmTitleCount'] = data.Title.apply(lambda x: x.count('+') + x.count('-') + x.count('*'))
    data['{Count'] = data.BodyMarkdown.apply(lambda x: x.count('{') + x.count('}'))
    data['[Count'] = data.BodyMarkdown.apply(lambda x: x.count('[') + x.count(']'))
    data['(Count'] = data.BodyMarkdown.apply(lambda x: x.count('(') + x.count(')'))
    return data

In [7]:
data = feature_add(data)
test = feature_add(test)

### Стемминг

Стемминг слов производился для одной из моделей (описаны функции, сам стеминг производится позже) 

In [14]:
def stem_str(s):
    st = SnowballStemmer("english")
    res = ' '.join([st.stem(word) for word in word_tokenize(s)])   
    return res.encode('ascii')

def stemming(data):
    data['TextStem'] = data.Text.apply(lambda x: stem_str(x))
    data['TitleStem'] = data.Title.apply(lambda x: stem_str(x))
    return data


### Перевод данных в формат Vowpal Wabbit

Две функции для перевода в формат Vowpal Wabbit текста и матрицы TF-IDF

In [15]:
def process_for_vw(s):
    textdata = feature_add(data)
    test = feature_add(test) = filter(lambda x: len(x) > 1, re.split("[^a-z]", s.lower()))
    text = set(text)
    text = ' '.join(text)
    return text


def itdf_to_vw(data):
    l = []
    t = sps.find(data)
    lens = run_len_encode_vec(t[0])
    ind = lens[0]
    lens = lens[1]
    set_of_texts = np.unique(t[0])
    i = -1
    for j in range(data.shape[0]):
        if j not in set_of_texts:
            l.append(' ')
            continue
        i += 1
        temp = t[1][lens[i]:lens[i + 1]]
        temp2 = t[2][lens[i]:lens[i + 1]]
        z = zip(temp, temp2)
        s = [str(index) + ':' + str(feature) + ' ' for (index, feature) in z]
        s = ' '.join(s)
        l.append(s)
    return l

In [16]:
data_itdf_vw = itdf_to_vw(data_itdf)
test_itdf_vw = itdf_to_vw(test_itdf)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:181: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 12327793 but corresponding boolean dimension is 12327792
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:181: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 5232525 but corresponding boolean dimension is 5232524


38254


## Обучение моделей в Vowpal Wabbit

В VW было обучено две модели. В обоих случаях использовалась логистическая регрессия. Одна построена на модели TF-IDF и некоторых признаках, другая построена на признаках и по множествам слов в текстах сообщений (т.е. учитывалось только наличие конкретного слова в тексте, но не учитывалось количество его встречаний). Для второй модели производился стемминг (для модели с TF-IDF не производился).



In [17]:
def save_to_vw_itdf(data, fname, data_itdf_vw, is_test):
    i = 0
    target = 0
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            if is_test == 0:
                if row.OpenStatus == 1:
                    target = 1
                else:
                    target = -1
            
            s = data_itdf_vw[i]
            i += 1
       
            if is_test == 0:
                big_text = '{0} |itdf {1}'
            else:
                big_text = '|itdf {1}'
            lens = '|len 1:{8} 2:{9} 3:{10} 4:{11} 5:{12} 6:{13}'
            spec = '|spec 1:{14} 2:{15}'
            nwords = '|nwords 1:{16} 2:{17}'
            nsent = '|nsent 1:{18} 2:{19} 3:{20} 4:{21} 5:{22}'
            exist = '|exist 1:{23} 2:{24} 3:{25} 4:{26} 5:{27} 6:{28} 7:{29}'
            not_ascii = '|noas 1:{30}'# 2:{31} 3:{32} 4:{33}'
            spec_word = '|spw 1:{34} 2:{35} 3:{36} 4:{37}'
            ratio = '|ratio 1:{38} 2:{39} 3:{40} 4:{41} 5:{42} 6:{43} 7:{44}'
            len2 = '|lentwo 1:{45} 2:{46} 3:{47} 4:{48} 5:{49}'
            line = '|line 1:{50} 2:{51}'
            first = '|first 1:{52} 2:{53}'

            feature = big_text + lens + spec + nwords + nsent + exist + not_ascii + spec_word + ratio + line + first + '\n'
            fout.write(feature.format(target, 
                    s,  ' ',
                    row.Tag1, row.Tag2, row.Tag3, row.Tag4, row.Tag5,
                    row.TextLen, row.CodeLen, row.TitleLen, row.Tag1Len, row.FirstTextLen, row.LastTextLen,
                    row.ReputationAtPostCreation, row.OwnerUndeletedAnswerCountAtPostTime,
                    row.NWordsInText, row.NWordsInTitle,
                    row.NSentenceInText, row.RatioQuestAll, row.RatioDotAll, row.RatioExclAll, row.FirstLetterInSentence,
                    row.TextExist, row.CodeExist, row.Tag1Exist, row.Tag2Exist, row.Tag3Exist, row.Tag4Exist, row.Tag5Exist,
                    row.RatioNotAsciiBody, row.RatioNotAsciiTitle, row.NotAsciiBody, row.NotAsciiTitle,
                    row.Thanks, row.I, row.UrlCount, row.DigitCount,
                    row.RatioTextCode, row.RatioFirstText, row.RatioLastText, row.RatioTextNWords, 
                                     row.RatioTextNSentence, row.RatioNWordsNSentence, row.RatioCodeNLines,
                    row.Tag1Len, row.Tag2Len, row.Tag3Len, row.Tag4Len, row.Tag5Len,
                    row.NLinesInText, row.NLinesInCode,
                    row.FirstIsText, row.FirstIsCode))



In [18]:
def save_to_vw(data, fname, is_test):
    i = -1
    target = 0
    with open(fname, 'w') as fout:
        for _, row in data.iterrows():
            if is_test == 0:
                if row.OpenStatus == 1:
                    target = 1
                else:
                    target = -1
        
            text = process_for_vw((row.TextStem))
            title = process_for_vw((row.TitleStem))


            if is_test == 0:
                big_text = '{0} |tit {1} |btex {2}'
            else:
                big_text = '|tit {1} |btex {2}'
            tags = '|tone {3} |ttwo {4} |tthree {5} |tfour {6} |tfive {7}'
            lens = '|len 1:{8} 2:{9} 3:{10} 4:{11} 5:{12} 6:{13}'
            spec = '|spec 1:{14} 2:{15}'
            nwords = '|nwords 1:{16} 2:{17}'
            nsent = '|nsent 1:{18} 2:{19} 3:{20} 4:{21} 5:{22}'
            exist = '|exist 1:{23} 2:{24} 3:{25} 4:{26} 5:{27} 6:{28} 7:{29}'
            not_ascii = '|noas 1:{30} 2:{31} 3:{32} 4:{33}'
            spec_word = '|spw 1:{34} 2:{35} 3:{36} 4:{37}'
            ratio = '|ratio 1:{38} 2:{39} 3:{40} 4:{41} 5:{42} 6:{43} 7:{44}'
            len2 = '|lentwo 1:{45} 2:{46} 3:{47} 4:{48} 5:{49}'
            line = '|line 1:{50} 2:{51}'
            first = '|f 1:{52} 2:{53}'

            feature = big_text + tags + lens + spec + nwords + nsent + exist + not_ascii + spec_word + ratio + line + first + '\n'
            
            fout.write(feature.format(target, 
                    title, text,
                    row.Tag1, row.Tag2, row.Tag3, row.Tag4, row.Tag5,
                    row.TextLen, row.CodeLen, row.TitleLen, row.Tag1Len, row.FirstTextLen, row.LastTextLen,
                    row.ReputationAtPostCreation, row.OwnerUndeletedAnswerCountAtPostTime,
                    row.NWordsInText, row.NWordsInTitle,
                    row.NSentenceInText, row.RatioQuestAll, row.RatioDotAll, row.RatioExclAll, row.FirstLetterInSentence,
                    row.TextExist, row.CodeExist, row.Tag1Exist, row.Tag2Exist, row.Tag3Exist, row.Tag4Exist, row.Tag5Exist,
                    row.RatioNotAsciiBody, row.RatioNotAsciiTitle, row.NotAsciiBody, row.NotAsciiTitle,
                    row.Thanks, row.I, row.UrlCount, row.DigitCount,
                    row.RatioTextCode, row.RatioFirstText, row.RatioLastText, row.RatioTextNWords, 
                                     row.RatioTextNSentence, row.RatioNWordsNSentence, row.RatioCodeNLines,
                    row.Tag1Len, row.Tag2Len, row.Tag3Len, row.Tag4Len, row.Tag5Len,
                    row.NLinesInText, row.NLinesInCode,
                    row.FirstIsText, row.FirstIsCode))



In [19]:
def calc_vw_qual():
    preds = pd.read_csv('pred.txt', header=None).iloc[:, 0].values
    target = test.OpenStatus.values
    print roc_auc_score(target, preds)
    return preds

In [20]:
def norm_preds(preds):
    preds1 = preds + np.abs(preds.min())
    preds2 = preds1 / preds1.max()
    return preds2

### Первая модель (TF-IDF)

In [21]:
save_to_vw_itdf(data, 'train.vw', data_itdf_vw, 0)
save_to_vw_itdf(test, 'test.vw', test_itdf_vw, 1)

In [34]:
!vw -d train.vw --loss_function=logistic -c -k -f model.vw -q ff -b 18 --passes 10 -l 0.08 --quiet

In [35]:
!vw -d test.vw --loss_function=logistic -i model.vw -t -p pred.txt  --quiet

In [39]:
preds = calc_vw_qual()

0.848511526301


In [88]:
preds = pd.read_csv('pred.txt', header=None).iloc[:, 0].values

Результат на старой тестовой выборке - 0.855506

In [40]:
preds = norm_preds(preds)

### Вторая модель (set of words)

In [41]:
data = stemming(data)
test = stemming(test)

In [42]:
save_to_vw(data, 'train1.vw', 0)
save_to_vw(test, 'test1.vw', 1)

In [43]:
!vw -d train1.vw --loss_function=logistic -c -k -f model.vw -q ff -b 18 --passes 10 -l 0.175 --quiet --l2 0.000000005

In [44]:
!vw -d test1.vw --loss_function=logistic -i model.vw -t -p pred.txt  --quiet

In [45]:
preds1 = calc_vw_qual()

0.847046668636


In [96]:
preds1 = pd.read_csv('pred.txt', header=None).iloc[:, 0].values

In [46]:
preds1 = norm_preds(preds1)

Совмещение моделей немного улучшает результаты

In [47]:
for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    print(alpha, roc_auc_score(test.OpenStatus.values, alpha * preds + (1 - alpha) * preds1))

(0.1, 0.85012707751632488)
(0.2, 0.85208664033809534)
(0.3, 0.85317123523324534)
(0.4, 0.85360130918588517)
(0.5, 0.85348813124513112)
(0.6, 0.85297560606265055)
(0.7, 0.85215258265527472)
(0.8, 0.85109816113690751)
(0.9, 0.84986812381009824)
(1, 0.84851152630104232)


### Random Forest

Также на TF-IDF и признаках был обучен Random Forest.

In [48]:
def my_final_RF (data, test):    
    y_train = data.OpenStatus
   
    feature_set_RF = ['RatioTextCode', 'TextLen', 'TitleLen', 'RatioTextNWords', 'ReputationAtPostCreation',
                 'NWordsInBody', 'RatioTextNSentence', 'RatioLastText', 'RatioFirstText', 'FirstTextLen',
                 'NWordsInText', 'LastTextLen', 'CodeLen', 'RatioDotAll', 'OwnerUndeletedAnswerCountAtPostTime',
                 'RatioCodeNLines', 'NLinesInCode', 'Tag2Len', 'NWordsInTitle', 'Tag1Len', 'FirstLetterInSentence',
                 'NLinesInText', 'DigitCount', 'Tag3Len', 'RatioQuestAll', 'NSentenceInText', 'I',
                     'Tag4Len', 'NQuest', 'CodeExist']    
    
    X_LR_train = data[feature_set_RF]
    X_LR_test = test[feature_set_RF]
    
    y_train = data.OpenStatus
    
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_LR_train, y_train)
    
    return clf.predict_proba(X_LR_test)[:, 1]

In [49]:
preds_rf = my_final_RF(data, test)

In [50]:
roc_auc_score(test.OpenStatus, preds_rf)

0.74720352486196462

### Композиция всех трёх моделей

In [51]:
alpha = 0.45
beta = 0.94
roc_auc_score(test.OpenStatus.values, beta * (alpha * preds + (1 - alpha) * preds1) + (1 - beta) * preds_rf)

0.85621834406552666

### Запись результатов

In [101]:
ind = test.PostId
preds = pd.DataFrame({"PostId": ind, "OpenStatus": final_preds})
preds = preds.set_index("PostId")
preds.to_csv('predictions2.csv')

### Итог:

Для финальной модели использовалась композиция трёх моделей: двух логистических регрессий и одного случайного леса. Итоговый score: 0.85237